In [9]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import pandas as pd
import sys
path_to_package = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(path_to_package)
from src.environment import Environment

In [18]:
df = pd.read_csv('../data/close.csv', index_col=0)
df = df.iloc[-50:, :3]

print(df.shape[0])

50


In [ ]:
env = Environment(stock_market_history=df,                
                  initial_cash_in_bank=10000,
                  buy_rate=0.01,
                  sell_rate=0.01,
                  limit_n_stocks=100,
                  buy_rule='most_first')

In [11]:
print(env._get_observation())
print(env.stock_market_history.head())
print(env._get_portfolio_value())

[10000.           170.94999695   108.36000061    84.25
     0.             0.             0.        ]
                   MMM         ABT       ABBV
2020-10-20  170.949997  108.360001  84.250000
2020-10-21  169.169998  105.930000  82.889999
2020-10-22  170.710007  109.480003  84.309998
2020-10-23  169.800003  107.790001  84.339996
2020-10-26  166.160004  109.000000  83.919998
10000.0


In [12]:
env.step(actions=np.array([.1, .1, .1]))

(array([6328.04402466,  169.16999817,  105.93000031,   82.88999939,
          10.        ,   10.        ,   10.        ]),
 -92.05599670410083,
 False,
 {'value_portfolio': 9907.9440032959})

In [13]:
env._get_portfolio_value()

9907.9440032959

In [14]:
env.step(actions=np.array([.1, -.1, -.2]))

(array([6488.74504013,  170.71000671,  109.48000336,   84.30999756,
          20.        ,    0.        ,    0.        ]),
 -4.998828887939453,
 False,
 {'value_portfolio': 9902.94517440796})

In [16]:
env._get_observation()

array([6488.74504013,  170.71000671,  109.48000336,   84.30999756,
         20.        ,    0.        ,    0.        ])